In [1]:
import requests
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import csv

Connect to Kafka

In [ ]:
producer = KafkaProducer(bootstrap_servers=['100.25.219.224:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

Start streaming

In [ ]:
MY_SQL_API_URL='http://your-api-url/endpoint'

def publish_to_kafka(file_path,producer=producer,chunk_size=1000):
    """ 
    Send each row to MySQl API Kafka and
    """
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        while True:
            chunk = []
            for _ in range(chunk_size):
                try:
                    chunk.append(next(reader))
                except StopIteration:
                    break
            if not chunk:
                break
            for row in chunk:
                # Convert row to JSON
                json_data = dumps(row)
                
                # Send row to MYSQL API
                try:
                    response = requests.post(MY_SQL_API_URL, json=json_data)
                    response.raise_for_status()
                    print(f"API response: {response.json()}")
                except Exception as e:
                    print(f"Error sending row to API: {e}")

                # Publish row to Kafka
                producer.send('invoice', value=bytes(json_data, 'utf-8'))
                sleep(1)
            producer.flush()
    producer.close()
    producer.close()

In [ ]:
publish_to_kafka("data/Iowa_Liquor_Sales.csv")

Manually flush data when done

In [ ]:
producer.flush()

Test

In [ ]:
with open("data/Iowa_Liquor_Sales.csv", 'r') as file:
    reader = csv.DictReader(file)
    while True:
        chunk = []
        for _ in range(1000):
            try:
                chunk.append(next(reader))
            except StopIteration:
                break
        if not chunk:
            break
        for row in chunk:
            print(row)